# Denver Coffee Shop (08/19/2022)

## Where to open a new coffee shop?

### 📖 Background
You are helping a client who owns coffee shops in Colorado. The company's coffee shops serve high-quality and responsibly sourced coffee,  pastries, and sandwiches. They operate three locations in Fort Collins and want to expand into Denver. 

Your client believes that the ideal location for a new store is close to affluent households, and the store appeals to the 20-35 year old demographic.
 
Your team collected geographical and demographic information about Denver's neighborhoods to assist the search. They also collected data for Starbucks stores in Denver. Starbucks and the new coffee shops do not compete for the same clients; the team included their location as a reference.

### 💾 The data
You have assembled information from three different sources ([locations](https://github.com/chrismeller/), [neighborhoods](http://data.denvergov.org), [demographics](https://www.census.gov/)):

#### Starbucks locations in Denver, Colorado
- "StoreNumber" - Store Number as assigned by Starbucks
- "Name" - Name identifier for the store
- "PhoneNumber" - Phone number for the store
- "Street 1, 2, and 3" - Address for the store
- "PostalCode" - Zip code of the store
- "Longitude, Latitude" - Coordinates of the store

#### Neighborhoods' geographical information
- "NBHD_ID" - Neighborhood ID (matches the census information)
- "NBHD_NAME" - Name of the statistical neighborhood
- "Geometry" - Polygon that defines the neighborhood

#### Demographic information
- "NBHD_ID" - Neighborhood ID (matches the geographical information)
- "NBHD_NAME' - Nieghborhood name
- "POPULATION_2010' - Population in 2010
- "AGE_ " - Number of people in each age bracket (< 18, 18-34, 35-65, and > 65)
- "NUM_HOUSEHOLDS" - Number of households in the neighborhood
- "FAMILIES" - Number of families in the neighborhood
- "NUM_HHLD_100K+" - Number of households with income above 100 thousand USD per year

_Starbucks locations were scrapped from the Starbucks store locator webpage by [Chris Meller](https://github.com/chrismeller/)._  
_Statistical Neighborhood information from the [City of Denver Open Data Catalog](http://data.denvergov.org), [CC BY 3.0](http://creativecommons.org/licenses/by/3.0/) license._      
_Census information from the [United States Census Bureau](https://www.census.gov/). Publicly available information._

### 💪 Challenge
Provide your client a list of neighborhoods in Denver where they should consider expanding. Include:
* A visualization of Denver's neighborhoods and the Starbucks store locations.
* Find the neighborhoods with the highest proportion of people in the target demographic.
* Select the top three neighborhoods where your client should focus their search.

## 💡 My Idea

If we are interested in the population of ages 20-35, we are most interested in neighborhoods with a higher population of this range (maybe person per sq area) and also with the most money to spend. Since we don't compete with Starbucks, we can possibly mimic a location selection strategy to Starbucks. To start simple, let's just find the neighborhoods with the highest age 20-35 population, and then subsort by salary.


In [49]:
import pandas as pd
import geopandas as gpd
import numpy as np

denver = pd.read_csv('./data/denver.csv')
den_df = pd.DataFrame(denver)
den_df

,StoreNumber,Name,PhoneNumber,Street1,Street2,Street3,PostalCode,Longitude,Latitude
0,27708-240564,Auraria Tivoli - Student Union,3035563700,900 Auraria Parkway,NaN,NaN,80204,-105.01,39.75
1,75828-94910,Grand Hyatt Denver,303-295-1234,1740 Welton St,NaN,NaN,802023940,-104.99,39.75
2,74105-23807,Safeway - Denver #1463,303-722-6117,2150 S Downing,NaN,NaN,802104528,-104.97,39.68
3,22871-102491,Safeway - Denver #1248,303-477-5091,2660 Federal Blvd,NaN,NaN,80211,-105.02,39.76
4,11105-100844,Highlands Garden Village,303-455-3382,4500 W 38th Ave,NaN,NaN,802122001,-105.05,39.77
5,75003-33060,King Soopers-Denver #29,303-832-5262,1155 E. 9th Ave.,NaN,Capitol Hill Center,802184802,-104.97,39.73
6,19695-165566,"Hampden & Santa Fe, Sheridan",3037611675,3500 A River Point Parkway,NaN,NaN,80110,-105.01,39.65
7,74108-17656,Safeway-Denver #1267,303-691-8870,6460 East Yale Ave,NaN,NaN,802227156,-104.91,39.67
8,18946-177247,Target Denver T-2820,303-481-2821,7777 E. Hampden Ave,NaN,NaN,80231,-104.90,39.65
9,5440-1319,5835 Leetsdale,(303) 388-8367,5835 Leetsdale Drive,NaN,NaN,802241222,-104.92,39.71


In [51]:
neighborhoods = gpd.read_file('./data/neighborhoods.shp')
n_gdf = gpd.GeoDataFrame(neighborhoods)
n_gdf

,NBHD_ID,NBHD_NAME,geometry
0,2,Auraria,"POLYGON ((-105.00042 39.74552, -105.00041 39.7..."
1,21,Cory - Merrill,"POLYGON ((-104.94070 39.69540, -104.94070 39.6..."
2,7,Belcaro,"POLYGON ((-104.94070 39.71156, -104.94069 39.7..."
3,70,Washington Park,"POLYGON ((-104.95931 39.71566, -104.95931 39.7..."
4,71,Washington Park West,"POLYGON ((-104.97342 39.68982, -104.97356 39.6..."
5,59,Speer,"POLYGON ((-104.97325 39.71840, -104.97323 39.7..."
6,13,Cherry Creek,"POLYGON ((-104.94070 39.71156, -104.94091 39.7..."
7,22,Country Club,"POLYGON ((-104.96017 39.72561, -104.96015 39.7..."
8,20,Congress Park,"POLYGON ((-104.94066 39.74013, -104.94066 39.7..."
9,14,City Park,"POLYGON ((-104.94062 39.75104, -104.94063 39.7..."


In [52]:
ppl = pd.read_csv('./data/census.csv')
ppl_df = pd.DataFrame(ppl)
ppl_df

,NBHD_ID,NBHD_NAME,POPULATION_2010,AGE_LESS_18,AGE_18_TO_34,AGE_35_TO_65,AGE_65_PLUS,NUM_HOUSEHOLDS,FAMILIES,NUM_HHLD_100K+
0,1,Athmar Park,8898.0,2649.0,2406.0,3112.0,731.0,2781.0,1964.0,425.0
1,2,Auraria,705.0,7.0,635.0,57.0,6.0,60.0,15.0,NaN
2,3,Baker,4879.0,745.0,1762.0,2037.0,335.0,2447.0,833.0,NaN
3,4,Barnum,6111.0,2015.0,1647.0,2089.0,360.0,1782.0,1289.0,120.0
4,5,Barnum West,5376.0,1579.0,1380.0,1824.0,593.0,1704.0,1207.0,NaN
5,6,Bear Valley,8889.0,2036.0,2375.0,3190.0,1288.0,3652.0,2285.0,862.0
6,7,Belcaro,4172.0,646.0,621.0,1913.0,992.0,2126.0,1162.0,NaN
7,8,Berkeley,8112.0,1305.0,2270.0,3513.0,1024.0,4049.0,1843.0,1510.0
8,9,Capitol Hill,14708.0,408.0,8274.0,5155.0,871.0,10856.0,1201.0,1223.0
9,10,CBD,3648.0,84.0,2288.0,1058.0,218.0,2363.0,303.0,NaN
